In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
links = [
"https://aliffchannel.com/netizen/",
"https://aliffchannel.com/travel/",
"https://aliffchannel.com/teknologi/",
"https://aliffchannel.com/lifestyle/",
"https://ering.com.my/category/islamik/"]

hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 


In [17]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "html.parser")

    div = soup.find_all('h2', attrs = {"class":"title"})
    
    if len(div) == 0:
        return

    for link in soup.find_all('h2', attrs = {"class":"title"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [ ]:
max_worker = 20


for url in links:
    
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    a = soup.find_all('a', attrs = {"class":"page-numbers"})
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])
    print(max_page)

    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


In [6]:
hrefs = list(set(hrefs))

In [7]:
len(hrefs)

10235

In [ ]:
with open(f'ering-link.json', 'a') as f:
    json.dump(hrefs, f)

In [5]:
with open('ering-link.json') as fopen:
    hrefs = json.load(fopen)

In [8]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('h1', class_="single-post-title").text
        h = soup.find('div', class_="entry-content clearfix single-post-content")
        p = [p.text for p in h.find_all("p") if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    
    return data

In [9]:
max_workers = 10

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('ering-2.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

 76%|███████▋  | 7824/10235 [43:24<08:13,  4.89it/s]  

HTTPSConnectionPool(host='ering.com.my', port=443): Max retries exceeded with url: /2019/08/promosi-merdeka-oppo-tawar-rebat-menarik-untuk-warga-emas-bermula-sekarang-1-september-2019/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1544d6a40>, 'Connection to ering.com.my timed out. (connect timeout=None)'))


100%|██████████| 10235/10235 [56:18<00:00,  3.03it/s] 
